In [1]:
import pandas as pd
import numpy as np
import csv
#import glob as glob
import os
from datetime import datetime, timedelta
from dateutil.parser import parse, tz

In [2]:
import sys
import os
import re
import fnmatch
try:
    _unicode = unicode
except NameError:
    # If Python is built without Unicode support, the unicode type
    # will not exist. Fake one.
    class _unicode(object):
        pass
__all__ = ["glob", "iglob"]
def glob(pathname):
    """Return a list of paths matching a pathname pattern.

    The pattern may contain simple shell-style wildcards a la
    fnmatch. However, unlike fnmatch, filenames starting with a
    dot are special cases that are not matched by '*' and '?'
    patterns.

    """
    return list(iglob(pathname))
def iglob(pathname):
    """Return an iterator which yields the paths matching a pathname pattern.

    The pattern may contain simple shell-style wildcards a la
    fnmatch. However, unlike fnmatch, filenames starting with a
    dot are special cases that are not matched by '*' and '?'
    patterns.

    """
    dirname, basename = os.path.split(pathname)
    if not has_magic(pathname):
        if basename:
            if os.path.lexists(pathname):
                yield pathname
        else:
            # Patterns ending with a slash should match only directories
            if os.path.isdir(dirname):
                yield pathname
        return
    if not dirname:
        for name in glob1(os.curdir, basename):
            yield name
        return
    # `os.path.split()` returns the argument itself as a dirname if it is a
    # drive or UNC path.  Prevent an infinite recursion if a drive or UNC path
    # contains magic characters (i.e. r'\\?\C:').
    if dirname != pathname and has_magic(dirname):
        dirs = iglob(dirname)
    else:
        dirs = [dirname]
    if has_magic(basename):
        glob_in_dir = glob1
    else:
        glob_in_dir = glob0
    for dirname in dirs:
        for name in glob_in_dir(dirname, basename):
            yield os.path.join(dirname, name)

# These 2 helper functions non-recursively glob inside a literal directory.
# They return a list of basenames. `glob1` accepts a pattern while `glob0`
# takes a literal basename (so it only has to check for its existence).

def glob1(dirname, pattern):
    if not dirname:
        dirname = os.curdir
    if isinstance(pattern, _unicode) and not isinstance(dirname, unicode):
        dirname = unicode(dirname, sys.getfilesystemencoding() or
                                   sys.getdefaultencoding())
    try:
        names = os.listdir(dirname)
    except os.error:
        return []
    if pattern[0] != '.':
        names = filter(lambda x: x[0] != '.', names)
    return fnmatch.filter(names, pattern)

def glob0(dirname, basename):
    if basename == '':
        # `os.path.split()` returns an empty basename for paths ending with a
        # directory separator.  'q*x/' should match only directories.
        if os.path.isdir(dirname):
            return [basename]
    else:
        if os.path.lexists(os.path.join(dirname, basename)):
            return [basename]
    return []


magic_check = re.compile('[*?[]')

def has_magic(s):
    return magic_check.search(s) is not None

In [56]:
directoryrfs='/Users/kaitlynlieschke/Documents/montalvin/data/2017_1*'
foldersrfs=glob(directoryrfs)
# Don't use San Pablo, Korematsu, Lighthouse (all show ~12 hourly noise from cell connection)

In [57]:
allfilesrfs=list()
for folder in foldersrfs:
   filesrfs=glob(os.path.join(directoryrfs,"*.csv"))
   allfilesrfs=allfilesrfs+filesrfs


In [58]:
#Create headernames for all columns in file
headernames=["BMP temperature","pressure","SHT temperature","RH","dew point temp","MiCS O3","MiCS CO","MiCS NO2",
             "Alpha O3","Alpha O3","Alpha CO","Alpha CO","Alpha NO","Alpha NO","Alpha NO2","Alpha NO2","high PM",
             "lowPM","%FS PM","CO2","GMP temperature","UTC time"]

In [59]:
dfrfs=pd.concat((pd.read_csv(file,names=headernames,na_values='-999') for file in allfilesrfs), ignore_index=True)


In [60]:
dfrfs.tail(5)

,BMP temperature,pressure,SHT temperature,RH,dew point temp,MiCS O3,MiCS CO,MiCS NO2,Alpha O3,Alpha O3.1,...,Alpha NO,Alpha NO.1,Alpha NO2,Alpha NO2.1,high PM,lowPM,%FS PM,CO2,GMP temperature,UTC time
238909,NaN,1013.31829,31.38,19.48633,5.34160,NaN,NaN,NaN,0.29381,0.27859,...,0.43063,0.41202,0.25467,0.26851,130112033.0,130142003.0,0.29483,432.4,30.6,2017-10-11 20:26:45
238910,NaN,1013.32757,31.37,19.49707,5.34136,NaN,NaN,NaN,0.29445,0.27823,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,428.7,30.7,2017-10-11 20:26:47
238911,NaN,1013.35125,31.37,19.42969,5.29148,NaN,NaN,NaN,0.29243,0.27694,...,0.43658,0.41321,0.25348,0.26970,124003340.0,130140302.0,3.30695,432.4,30.6,2017-10-11 20:26:52
238912,NaN,1013.33715,31.34,19.39453,5.24083,NaN,NaN,NaN,0.29344,0.27768,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,433.5,30.6,2017-10-11 20:26:55
238913,NaN,1013.31103,31.33,19.37109,5.21522,NaN,NaN,NaN,0.29252,0.27813,...,0.43484,0.41102,0.25220,0.26970,124010412.0,130134314.0,3.24731,436.1,30.5,2017-10-11 20:27:00


In [34]:
# df1=df1.dropna(axis=0, how='any')
# df2=df2.dropna(axis=0, how='any')
# df3=df3.dropna(axis=0, how='any')
# df4=df4.dropna(axis=0, how='any')
# df5=df5.dropna(axis=0, how='any')
# df6=df6.dropna(axis=0, how='any')
# df7=df7.dropna(axis=0, how='any')
# df8=df8.dropna(axis=0, how='any')
# df9=df9.dropna(axis=0, how='any')
# df10=df10.dropna(axis=0, how='any')
# df11=df11.dropna(axis=0, how='any')
# df12=df12.dropna(axis=0, how='any')
# df13=df13.dropna(axis=0, how='any')
# df14=df14.dropna(axis=0, how='any')
# df15=df15.dropna(axis=0, how='any')
# Is it bad to remove NaN rows before concatenating all of the columns? The dates won't match up?

In [61]:
dfrfs=dfrfs[['%FS PM','SHT temperature','RH','CO2','UTC time']] 

In [9]:
dfrfs=dfrfs[(dfrfs['UTC time'] >= '2017-10-19') & (dfrfs['UTC time'] <= '2017-08-10')]


In [62]:
dfrfs=dfrfs.reset_index()
dfrfs=dfrfs[['%FS PM','SHT temperature','RH','CO2','UTC time']] 


In [63]:
dfrfs.head()

,%FS PM,SHT temperature,RH,CO2,UTC time
0,1.68135,35.06,28.80176,429.9,2017-10-01 00:00:01
1,NaN,35.08,28.76758,427.7,2017-10-01 00:00:03
2,6.39861,35.06,28.85840,430.1,2017-10-01 00:00:09
3,NaN,35.02,28.93457,427.6,2017-10-01 00:00:11
4,0.00000,35.04,28.94727,430.1,2017-10-01 00:00:16


In [64]:
dfrfs['datetime']=dfrfs['UTC time'].apply(lambda row: parse(row)) 


In [65]:
dfrfs.head(5)

,%FS PM,SHT temperature,RH,CO2,UTC time,datetime
0,1.68135,35.06,28.80176,429.9,2017-10-01 00:00:01,2017-10-01 00:00:01
1,NaN,35.08,28.76758,427.7,2017-10-01 00:00:03,2017-10-01 00:00:03
2,6.39861,35.06,28.85840,430.1,2017-10-01 00:00:09,2017-10-01 00:00:09
3,NaN,35.02,28.93457,427.6,2017-10-01 00:00:11,2017-10-01 00:00:11
4,0.00000,35.04,28.94727,430.1,2017-10-01 00:00:16,2017-10-01 00:00:16


In [66]:
dfrfs['day'] = dfrfs['datetime'].apply(lambda x: x.day)
dfrfs['month'] = dfrfs['datetime'].apply(lambda x: x.month)
dfrfs['hour']=dfrfs['datetime'].apply(lambda x: x.hour)
dfrfs['year']=dfrfs['datetime'].apply(lambda x: x.year)
dfrfs = dfrfs.drop('datetime', axis=1)
dfrfs=dfrfs.apply(lambda x: pd.to_numeric(x, errors='coerce'))
dfrfs_t = dfrfs.groupby(['day','month','hour','year'], as_index=False,sort=False)
aggregation = {'SHT temperature':'mean',
               'RH':'mean',
               'CO2':'mean',
               '%FS PM':'mean'
                 }
dfrfs_t = dfrfs_t.agg(aggregation)
dfrfs_t['datetime'] = dfrfs_t[['year', 'month', 'day', 'hour']].apply(lambda s : datetime(*s),axis = 1)


In [67]:
dfrfs_t.head()

,day,month,hour,year,SHT temperature,RH,CO2,%FS PM,datetime
0,1,10,0,2017,35.239935,28.716227,431.254962,0.625288,2017-10-01 00:00:00
1,1,10,1,2017,29.400632,38.150392,455.952288,0.761161,2017-10-01 01:00:00
2,1,10,2,2017,25.779422,46.015990,475.483642,0.878156,2017-10-01 02:00:00
3,1,10,3,2017,24.512050,47.884998,478.431952,0.879143,2017-10-01 03:00:00
4,1,10,4,2017,23.580720,49.597080,486.950055,0.834242,2017-10-01 04:00:00


In [68]:
dfrfs_t=dfrfs_t[['datetime','SHT temperature','RH','CO2','%FS PM']] 


In [69]:
dfrfs_t['ug/m3 PM']=(dfrfs_t['%FS PM'] * 3.5)+5


In [70]:
dfrfs_t.tail()

,datetime,SHT temperature,RH,CO2,%FS PM,ug/m3 PM
256,2017-10-11 16:00:00,18.602268,50.049357,495.492795,1.123436,8.932026
257,2017-10-11 17:00:00,20.310033,43.376809,485.046514,1.172195,9.102684
258,2017-10-11 18:00:00,25.058157,31.932785,454.816140,1.209338,9.232682
259,2017-10-11 19:00:00,28.630959,24.594054,439.583115,1.373047,9.805665
260,2017-10-11 20:00:00,30.448257,20.340951,436.880630,1.553006,10.435521


In [71]:
print(dfrfs_t.shape)

(261, 6)


In [72]:
######### FIGURE OUT WHAT TO DO WITH DIFFERENT SIZED DATAFRAMES HERE!!!

In [73]:
rfsdf = dfrfs_t


In [74]:
rfsdf.head(10)

,datetime,SHT temperature,RH,CO2,%FS PM,ug/m3 PM
0,2017-10-01 00:00:00,35.239935,28.716227,431.254962,0.625288,7.188509
1,2017-10-01 01:00:00,29.400632,38.150392,455.952288,0.761161,7.664062
2,2017-10-01 02:00:00,25.779422,46.015990,475.483642,0.878156,8.073547
3,2017-10-01 03:00:00,24.512050,47.884998,478.431952,0.879143,8.077001
4,2017-10-01 04:00:00,23.580720,49.597080,486.950055,0.834242,7.919847
5,2017-10-01 05:00:00,23.764242,45.882864,471.438604,0.733663,7.567822
6,2017-10-01 06:00:00,23.491885,45.371510,473.881373,0.534590,6.871063
7,2017-10-01 07:00:00,23.061832,45.777468,475.324864,0.601770,7.106195
8,2017-10-01 08:00:00,22.401145,46.455920,476.893348,0.540378,6.891323
9,2017-10-01 09:00:00,22.086383,45.670622,473.587255,0.449400,6.572900


In [75]:
rfsdf.columns = ['UTCdatetime','LBLtemp', 'LBLRH', 'LBLCO2', 'LBLFSPM', 'LBLUGPM']


In [76]:
# Change timezone from UTC to Local Time (GMT-7)
# Create new dataframe to contain local times, with length of old dataframe
localrfs = pd.DataFrame(index=rfsdf.index.values, columns=['localtime'])
# Loop through all dates in old dataframe and convert to local date in new dataframe
for dateutc in rfsdf.index.values:
    from_zone = tz.gettz('UTC')
    to_zone = tz.gettz('America/Los_Angeles')
    utc = rfsdf['UTCdatetime'][dateutc]
    utc = utc.replace(tzinfo=from_zone)
    localrfs['localtime'][dateutc]=utc.astimezone(to_zone)

In [77]:
localrfs.head()

,localtime
0,2017-09-30 17:00:00-07:00
1,2017-09-30 18:00:00-07:00
2,2017-09-30 19:00:00-07:00
3,2017-09-30 20:00:00-07:00
4,2017-09-30 21:00:00-07:00


In [78]:
rfsfinaldf = pd.concat([localrfs,rfsdf],ignore_index=True, axis=1)
rfsfinaldf.columns = ['Localdatetime','UTCdatetime','RFStemp', 'RFSRH', 'RFSCO2', 'RFSFSPM','RFSUGPM']


In [79]:
rfsfinaldf.head()

,Localdatetime,UTCdatetime,RFStemp,RFSRH,RFSCO2,RFSFSPM,RFSUGPM
0,2017-09-30 17:00:00-07:00,2017-10-01 00:00:00,35.239935,28.716227,431.254962,0.625288,7.188509
1,2017-09-30 18:00:00-07:00,2017-10-01 01:00:00,29.400632,38.150392,455.952288,0.761161,7.664062
2,2017-09-30 19:00:00-07:00,2017-10-01 02:00:00,25.779422,46.015990,475.483642,0.878156,8.073547
3,2017-09-30 20:00:00-07:00,2017-10-01 03:00:00,24.512050,47.884998,478.431952,0.879143,8.077001
4,2017-09-30 21:00:00-07:00,2017-10-01 04:00:00,23.580720,49.597080,486.950055,0.834242,7.919847


In [80]:
rfsfinaldf.to_csv(path_or_buf='beacon_mta_oct.csv', sep=',', na_rep='NaN', index=False)
